# StarGAN  - To Recognize Person on the video based on the image we have.

Currently testing the algorithm to recongnize the  celebrities from youtube video based  on wiki image.

## Wiki Dataset

### Installing Necessary Package

Downloading the necessary package and create folder for storing the wiki data.

In [1]:
!pip install wikipedia




  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=25536751381d58c16171d61e9acab0a5d6736edfee47c0f5e4e7025d7ac2258c
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


Creating the folder 


In [59]:
%cd /content
!rm -r celeb
!mkdir /content/celeb
%cd /content/celeb

/content
rm: cannot remove 'celeb': No such file or directory
/content/celeb


### Importing necessary packages.

In [0]:
import wikipedia
import urllib.request
import numpy as np
import os



        

### GET Wiki Images

In [0]:
def get_wikiImage(title , nickname):
  mainPage = wikipedia.page(title)
  ext = (".jpg",".png")
  try: 
    wikipage = mainPage
    print("Page Title: %s" % wikipage.title)
    # print ("Page URL: %s" % wikipage.url)
    # print (" - Main Image: %s" % wikipage.images[0])
    for count,image in enumerate(wikipage.images):
      # print('the images are ' , image);
      baseName = os.path.basename(image)
      # print(baseName)
      if image.endswith(ext):
        urllib.request.urlretrieve(image, nickname + '(' + str(count) +')' + "-" + baseName)
    print ("")
  except:
    print("Could not get images form the page")
    pass
  wikiPage = wikipedia.page(title).links
  
  print(len(wikiPage))
  np.random.shuffle(wikiPage)
  for page in wikiPage[:5]:
      try: 
        try:
            wikipage = wikipedia.page(page)
        except wikipedia.DisambiguationError as e:
            s = random.choice(e.options)
            wikipage = wikipedia.page(s)
        # wikipage = wikipedia.page(page)
        # print("Page Title: %s" % wikipage.title)
        # print ("Page URL: %s" % wikipage.url)
        # print (" - Main Image: %s" % wikipage.images[0])
        for count,image in enumerate(wikipage.images):
          baseName = os.path.basename(image)
          if image.endswith(ext):
              urllib.request.urlretrieve(image,nickname +'-' + page+str(count)+".jpg")
              urllib.request.urlretrieve(image, nickname + '(' + str(count) +')' + "-" + baseName)
        print ("")
      except:
        print("Could not get images form the page")
        pass

Get images from wiki url and download images.

**Note -You can pass any  page title on the function to get images from its link.**

In [60]:
get_wikiImage("List_of_Game_of_Thrones_characters" , "GOT")

Page Title: List of Game of Thrones characters

508



Could not get images form the page



In [61]:
get_wikiImage("List_of_Friends_and_Joey_characters" , 'Friends')

Page Title: List of Friends and Joey characters

376


Could not get images form the page
Could not get images form the page



In [62]:
get_wikiImage("List_of_The_Big_Bang_Theory_and_Young_Sheldon_characters" , 'BBTY')

Page Title: List of The Big Bang Theory and Young Sheldon characters

406







## Video Dataset

### Installing necessary package 


inspired by the code https://github.com/ahmetozlu/face_recognition_crop

In [8]:
!pip install pytube3
  

In [9]:
!pip install autocrop

     |████████████████████████████████| 163kB 1.4MB/s 
     |████████████████████████████████| 28.2MB 1.2MB/s 
  Created wheel for autocrop: filename=autocrop-1.0.2-cp36-none-any.whl size=157696 sha256=4af100751792f6cb6fc355ac548d9f90154d89cc29d97543ebf88a2e26cdf79f
  Stored in directory: /root/.cache/pip/wheels/8e/5a/8b/e173ae548ba963c0bd47500b8631196acc878e4666ca121ebc
Successfully built autocrop
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [10]:
!pip install face_recognition

     |████████████████████████████████| 100.2MB 93kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566172 sha256=d28cd848232243a102b3a6fce99133ac84220a9d7485509b2ec491afd3cfad77
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


### Importing the necessary package

In [11]:
%cd /content
!rm -r video
!mkdir video

/content
rm: cannot remove 'video': No such file or directory


In [0]:
import urllib
from bs4 import BeautifulSoup
from pytube import YouTube
import face_recognition

import cv2
import os
import glob

### getting url from youtube video search list

In [0]:
def get_urls(text, limit=10):
  query = urllib.parse.quote(text)
  url = "https://www.youtube.com/results?search_query=" + query
  response = urllib.request.urlopen(url)
  html = response.read()
  soup = BeautifulSoup(html, 'html.parser')
  urls = []
  for i, vid in enumerate(soup.findAll(attrs={'class':'yt-uix-tile-link'})):
      if i < limit:
          urls.append('https://www.youtube.com' + vid['href'])
  print(f"Found {len(urls)} video links for {text}")
  return urls

### Download Video

In [0]:

def download_video(url, path=None, max_duration=36000):
  try:
    yt = YouTube(url)
    # duration = int(yt.player_config_args['player_response']['streamingData']['formats'][0]['approxDurationMs'])
    duration = yt.length;
    print("The duration is " , duration);
    if duration < max_duration:
        yt = yt.streams.filter(file_extension='mp4').first()
        out_file = yt.download(path)
        file_name = out_file.split("\\")[-1]
        print(f"Downloaded {file_name} correctly!")
    else:
        print(f"Video {url} too long")
  except Exception as exc:
      print(f"Download of {url} did not work because of {exc}...")

###Labeling the folder 

In [0]:
def max_label(name, folder):
    '''Look at a folder and check the files with pattern "name_###.jpg" to extract the
    largest label present.'''
    
    path_pattern = os.path.join(folder, name + "_*.jpg")
    existing_files = glob.glob(path_pattern)
    if not existing_files:
        biggest_label = 0
    else:
        existing_labels = map(lambda s: int(s.split('_')[-1].split('.')[0]), existing_files)
        biggest_label = max(existing_labels)
    return biggest_label

### Extract Image 

In [0]:
def extract_images_from_video(video, folder=None, delay=30, name="video", max_images=200, silent=False):    
    vidcap = cv2.VideoCapture(video)
    count = 0
    num_images = 0
    if not folder:
        folder = os.getcwd()
    label = max_label(name, folder)
    success = True
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    
    while success and num_images < max_images:
        success, image = vidcap.read()
        num_images += 1
        label += 1
        # print("the image is " ,image)
        if image is not None:
          # Display the resulting frame
         #  cv2.imshow('frame',image)
          file_name = name + "_" + str(label) + ".jpg"
          path = os.path.join(folder, file_name)
          cv2.imwrite(path, image)
          if cv2.imread(path) is None:
              os.remove(path)
          else:
            if not silent:
                print(f'Image successfully written at {path}')
        else:
            print ("Frame is None....")
            break
        
        count += delay*fps
        vidcap.set(1, count)
    
    cv2.destroyAllWindows()
    vidcap.release()

In [0]:

def extract_images_from_word(text, download=True, delete_video=False, image_delay=30, 
                             num_urls=15, max_images=300, name="video", max_duration=36000, silent=False):
    if not os.path.exists(name):
        os.mkdir(name)
    
    if(download):
      urls = get_urls(text, num_urls)
      for url in urls:
          download_video(url, max_duration=max_duration)
    for i, video in enumerate(glob.glob("*.mp4")):
        print("the image video url " , video)
        extract_images_from_video(video, folder=name, delay=image_delay, name=name, max_images=max_images, silent=silent)
        if delete_video:
            os.remove(video)

### Implementing video extractor

In [18]:
extract_images_from_word('Freinds and Big bang theory' , True)
extract_images_from_word('money heist' , True)

Found 15 video links for Freinds and Big bang theory
Download of https://www.youtube.com/channel/UCQGNGhpDq-HGBKaP4jfez5Q did not work because of get_ytplayer_config: could not find match for config_patterns...
Download of https://www.youtube.com/channel/UC3-DfztgFS4vMwjQPUB7SPg did not work because of get_ytplayer_config: could not find match for config_patterns...
The duration is  541
Downloaded /content/10 Scenes From Big Bang Theory Which Are Copied From FRIENDS.mp4 correctly!
The duration is  374
Downloaded /content/Ross Geller Vs Howard Wolowitz Reaction to pregnancy Friends Vs The big bang theory.mp4 correctly!
The duration is  299
Downloaded /content/The Big Bang Theory Cast Interview - Tribute To Director James Burrows.mp4 correctly!
The duration is  900
Downloaded /content/2016 Friends Reunion - EXTENDED!!!.mp4 correctly!
The duration is  51
Downloaded /content/Kaley Cuoco Freaks Out Over Friends Reunion See Her Pic With the Cast!.mp4 correctly!
The duration is  138
Downloade

In [19]:
extract_images_from_word('Game of thrones' , True)

Found 15 video links for Game of thrones
Download of https://www.youtube.com/channel/UCow8-ZftRoZelY710tvO45Q did not work because of get_ytplayer_config: could not find match for config_patterns...
The duration is  1093
Downloaded /content/Best of Game of Thrones - Most Badass Scenes Compilation.mp4 correctly!
The duration is  542
Downloaded /content/Arya Stark Kills The Night King  Game of Thrones S08E03.mp4 correctly!
The duration is  400
Downloaded /content/The Entire Game Of Thrones Timeline Explained.mp4 correctly!
The duration is  152
Downloaded /content/Game of Thrones Season 7 Episode 4 Brienne and Arya (HBO).mp4 correctly!
The duration is  462
Downloaded /content/Questions left Unanswered after the Game of Thrones Finale.mp4 correctly!
The duration is  723
Downloaded /content/All of the Tribes in Westeros (Game of Thrones).mp4 correctly!
The duration is  1089
Downloaded /content/Game of Thrones - Best Scenes Compilation HD.mp4 correctly!
The duration is  1414
Downloaded /cont

### Zipping and downloading the image from video


Note : you dont need to run this block if you are not downloading the images extracted. This block was used to check the extracted images on local system.

In [20]:
!zip -r /content/images.zip /content/video

  adding: content/video/ (stored 0%)
  adding: content/video/video_1048.jpg (deflated 0%)
  adding: content/video/video_775.jpg (deflated 1%)
  adding: content/video/video_21.jpg (deflated 0%)
  adding: content/video/video_620.jpg (deflated 0%)
  adding: content/video/video_884.jpg (deflated 0%)
  adding: content/video/video_431.jpg (deflated 1%)
  adding: content/video/video_935.jpg (deflated 1%)
  adding: content/video/video_1123.jpg (deflated 1%)
  adding: content/video/video_316.jpg (deflated 1%)
  adding: content/video/video_1052.jpg (deflated 1%)
  adding: content/video/video_75.jpg (deflated 0%)
  adding: content/video/video_125.jpg (deflated 0%)
  adding: content/video/video_223.jpg (deflated 0%)
  adding: content/video/video_37.jpg (deflated 0%)
  adding: content/video/video_324.jpg (deflated 0%)
  adding: content/video/video_955.jpg (deflated 2%)
  adding: content/video/video_1098.jpg (deflated 10%)
  adding: content/video/video_390.jpg (deflated 1%)
  adding: content/video/v

In [21]:
!zip -FF images.zip --out new-images.zip

Fix archive (-FF) - salvage what can
 Found end record (EOCDR) - says expect single disk archive
Scanning for entries...
 copying: content/video/  (0 bytes)
 copying: content/video/video_1048.jpg  (56525 bytes)
 copying: content/video/video_775.jpg  (43029 bytes)
 copying: content/video/video_21.jpg  (73935 bytes)
 copying: content/video/video_620.jpg  (48069 bytes)
 copying: content/video/video_884.jpg  (69067 bytes)
 copying: content/video/video_431.jpg  (67592 bytes)
 copying: content/video/video_935.jpg  (34902 bytes)
 copying: content/video/video_1123.jpg  (93582 bytes)
 copying: content/video/video_316.jpg  (28849 bytes)
 copying: content/video/video_1052.jpg  (42247 bytes)
 copying: content/video/video_75.jpg  (69525 bytes)
 copying: content/video/video_125.jpg  (62818 bytes)
 copying: content/video/video_223.jpg  (44541 bytes)
 copying: content/video/video_37.jpg  (155561 bytes)
 copying: content/video/video_324.jpg  (52803 bytes)
 copying: content/video/video_955.jpg  (38757 b

## AUTOCROPPER

crops just the way we need it to but only works for 20 - 30 % of the images

It used to detect faces from the youtube dataset and wiki dataset , then crop only faces to get new dataset as we need for StarGan

In [22]:
%cd /content/
!rm -r  autocrop
!mkdir autocrop
%cd autocrop/
!mkdir pics_wiki
!mkdir crop_wiki
!mkdir reject_wiki
!mkdir pics_yt
!mkdir crop_yt
!mkdir reject_yt

/content
rm: cannot remove 'autocrop': No such file or directory
/content/autocrop


### Copying youtube dataset

In [58]:
!cp /content/video/* /content/autocrop/pics_yt/

cp: cannot stat '/content/video/*': No such file or directory


### Copying wiki dataset

In [0]:
!cp /content/celeb/* /content/autocrop/pics_wiki/

### Cropping all dataset images.

#### Cropping youtube data


In [25]:
!autocrop -i pics_yt -o crop_yt -w 800 -H 800 -r reject_yt

Processing images in folder: /content/autocrop/pics_yt
Face detected:    /content/autocrop/crop_yt/video_1048.jpg
No face detected: /content/autocrop/reject_yt/video_775.jpg
No face detected: /content/autocrop/reject_yt/video_21.jpg
Face detected:    /content/autocrop/crop_yt/video_620.jpg
No face detected: /content/autocrop/reject_yt/video_884.jpg
Face detected:    /content/autocrop/crop_yt/video_431.jpg
No face detected: /content/autocrop/reject_yt/video_935.jpg
No face detected: /content/autocrop/reject_yt/video_1123.jpg
No face detected: /content/autocrop/reject_yt/video_316.jpg
Face detected:    /content/autocrop/crop_yt/video_1052.jpg
No face detected: /content/autocrop/reject_yt/video_75.jpg
No face detected: /content/autocrop/reject_yt/video_125.jpg
No face detected: /content/autocrop/reject_yt/video_223.jpg
No face detected: /content/autocrop/reject_yt/video_37.jpg
Face detected:    /content/autocrop/crop_yt/video_324.jpg
No face detected: /content/autocrop/reject_yt/video_955

#### Cropping wiki data 

In [64]:
!autocrop -i pics_wiki -o crop_wiki -w 800 -H 800 -r reject_wiki

usage: autocrop [-h] [-i INPUT] [-o OUTPUT] [-r REJECT] [-w WIDTH] [-H HEIGHT]
                [-v] [--no-confirm] [--facePercent FACEPERCENT]
autocrop: error: argument -i/--input: Input folder does not exist


**Zipping and Downloading the cropped images**

Note : dont run this block of code if you dont want to download the images.

**Downloading Youtube cropped data**

In [0]:
# !zip -r /content/autocrop/cropped_yt.zip /content/autocrop/crop_yt/

# from google.colab import files
# files.download("/content/autocrop/cropped_yt.zip")

**Downloading Wiki cropped data**

In [0]:
# !zip -r /content/autocrop/cropped_wiki.zip /content/autocrop/crop_wiki/

# from google.colab import files
# files.download("/content/autocrop/cropped_wiki.zip")

## Implementation of StarGan

In [29]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 103kB/s 
ERROR: tensorflow 2.2.0rc2 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [30]:
!pip show tensorflow

Name: tensorflow
Version: 2.2.0rc2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: grpcio, gast, opt-einsum, six, keras-preprocessing, astunparse, scipy, protobuf, termcolor, absl-py, google-pasta, wrapt, numpy, tensorflow-estimator, tensorboard, h5py, wheel
Required-by: fancyimpute


In [31]:
pip install tensorflow==1.13.2

     |████████████████████████████████| 92.6MB 108kB/s 
     |████████████████████████████████| 3.2MB 57.5MB/s 
     |████████████████████████████████| 368kB 52.2MB/s 
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc2


In [32]:
%cd /content
!rm -r stargan-tensorflow


!git config --global user.name "sanjay adhikari , reuben francis , gautam meenakashi , bhavana ramanna"
!git config --global user.email "<dummy_name>@students.mq.edu.au"

!git clone https://gitlab.com/reubenbf/stargan-tensorflow.git
%cd /content/stargan-tensorflow

/content
rm: cannot remove 'stargan-tensorflow': No such file or directory
Cloning into 'stargan-tensorflow'...
remote: Enumerating objects: 204583, done.
remote: Counting objects: 100% (204583/204583), done.
remote: Compressing objects: 100% (204547/204547), done.
remote: Total 204583 (delta 34), reused 204577 (delta 28), pack-reused 0
Receiving objects: 100% (204583/204583), 3.12 GiB | 14.00 MiB/s, done.
Resolving deltas: 100% (34/34), done.
Checking out files: 100% (204641/204641), done.
/content/stargan-tensorflow


### STARGAN for generating new data.

#### Youtube

**Generating new image given the original image based on age , gender , and hair change.(i.e.) Dataset to detect disguise**

In [0]:
!rm -r /content/stargan-tensorflow/dataset/celebA/test
!mkdir /content/stargan-tensorflow/dataset/celebA/test
!cp  /content/autocrop/crop_yt/*  /content/stargan-tensorflow/dataset/celebA/test




In [0]:
!cp  /content/autocrop/crop_yt/*.jpg /content/stargan-tensorflow/New\ Dataset\ /dataset

In [35]:
%cd /content/stargan-tensorflow/
!rm -r results/StarGAN_celebA_wgan-gp_6resblock_6dis/images
!python main.py --phase test


/content/stargan-tensorflow
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWa

**Creating the folder to store**

In [36]:
%cd /content 
!mkdir result_data
!mkdir result_data/wiki
!mkdir result_data/yt

/content


##### Moving the generated images 

In [37]:
%cd /content/stargan-tensorflow/
!mv results/StarGAN_celebA_wgan-gp_6resblock_6dis/images/* /content/result_data/yt

/content/stargan-tensorflow


#### Wiki Data

In [0]:
# !rm -r /content/stargan-tensorflow/dataset/celebA/test
# !mkdir /content/stargan-tensorflow/dataset/celebA/test
# !cp  /content/autocrop/crop_wiki/* /content/stargan-tensorflow/dataset/celebA/test




In [0]:
# %cd /content/stargan-tensorflow/
# !rm -r results/StarGAN_celebA_wgan-gp_6resblock_6dis/images
# !python main.py --phase test

##### Moving the generated images 

In [0]:
# %cd /content/stargan-tensorflow/
# !mv results/StarGAN_celebA_wgan-gp_6resblock_6dis/images/* /content/result_data/wiki

### Merging the original images and generated images.

In [70]:
%cd /content
!rm -r  face_data
!mkdir face_data
!mkdir face_data/wiki
!mkdir face_data/yt

/content


In [71]:
%cd /content
!cp /content/autocrop/crop_yt/* face_data/yt
!cp /content/autocrop/crop_wiki/* face_data/wiki
!cp result_data/yt/* face_data/yt 
# !cp result_data/wiki/* face_data/wiki




/content


## Face Recog

In [42]:
!pip install face_recognition

### Detect Faces

In [0]:
%cd /content 

import glob 

import pandas as pd 

wiki_image_path = '/content/face_data/wiki/'
youtube_image_path = '/content/face_data/yt/'

wiki_image_list = glob.glob(wiki_image_path +'*.jpg')
youtube_image_list = glob.glob(youtube_image_path +'*.jpg');


rDf = pd.DataFrame(columns=['known_faces', 'known_face_encoding', 'matching_unknown_faces'])



results = []
for wiki , yt in zip(wiki_image_list , youtube_image_list):
  print(wiki)
  name = wiki.split('/')[-1]
  print(name)
  image = face_recognition.load_image_file(path)
  dItem = {"known_faces": ''  , "unknown_faces":[]}
  faces = face_recognition.face_locations(image)
  dItem['known_faces'] = name
  # if(len(faces) > 1):
  #   dItem['known_faces'] = name 
  # else:
  #   dItem['known_faces']=''
  try:

    known = face_recognition.face_encodings(image)[0]  
   
  except IndexError:
    known = []
  

  try:
    unknown_encoding = face_recognition.face_encodings(image)[0]
  except IndexError:
    unknown_encoding = []

  result = face_recognition.compare_faces([known], unknown_encoding)

  results.append(result)

  rDf = rDf.append(dItem , ignore_index=True)







  




/content
/content/face_data/wiki/GOT(65)-Lena_Headey_Primetime_Emmy_Awards_2014.jpg
GOT(65)-Lena_Headey_Primetime_Emmy_Awards_2014.jpg
/content/face_data/wiki/GOT(79)-Natalie_Dormer_2014.jpg
GOT(79)-Natalie_Dormer_2014.jpg
/content/face_data/wiki/GOT(68)-Lucian_Msamati_%282010%29_headshot.jpg
GOT(68)-Lucian_Msamati_%282010%29_headshot.jpg
/content/face_data/wiki/Friends(33)-Teri_Garr_-_signed.jpg
Friends(33)-Teri_Garr_-_signed.jpg
/content/face_data/wiki/GOT(34)-Hannah_Murray_-_IFFR_2015-1.jpg
GOT(34)-Hannah_Murray_-_IFFR_2015-1.jpg
/content/face_data/wiki/GOT(0)-Rosabell_Laurenti_Sellers.jpg
GOT(0)-Rosabell_Laurenti_Sellers.jpg
/content/face_data/wiki/Friends(29)-Robin_Williams_%286451536411%29_%28cropped%29.jpg
Friends(29)-Robin_Williams_%286451536411%29_%28cropped%29.jpg
/content/face_data/wiki/Friends-Smelly Cat0.jpg
Friends-Smelly Cat0.jpg
/content/face_data/wiki/BBTY(25)-Melissa_Rauch_at_PaleyFest_2013_%28edited%29.jpg
BBTY(25)-Melissa_Rauch_at_PaleyFest_2013_%28edited%29.jpg
/co

In [0]:
rDf.head(1)

In [0]:
print(results)

In [0]:
aDf.head()